In [ ]:
import os
import re
import shutil
from pathlib import Path
from unidecode import unidecode

In [ ]:
path = os.getcwd()
PATH_DFS = path + "/dfs"

In [ ]:
entries = os.listdir(PATH_DFS)
files = [f for f in entries]

In [ ]:
tipo_inst = {
    "Conglomerados_Financeiros_e_Instituições_Independentes": "cong_fin_indep",
    "Conglomerados_Prudenciais_e_Instituições_Independentes": "cong_pru_indep",
    "Instituições_Individuais": "inst_individuais",
    "Instituições_com_Operações_de_Câmbio": "inst_cambio"
}

In [ ]:
arq_por_inst = {
    short_name: [f for f in files if long_name in f]
    for long_name, short_name in tipo_inst.items()
}

In [ ]:
arq_por_inst["legado"] = [    
    f for f in files if not any(long_name in f for long_name in tipo_inst.keys())
]

In [ ]:
# agora, separar arquivos por tipo de instituicao
for k, v in arq_por_inst.items():
    path_inst = PATH_DFS + "/" + k
    os.mkdir(path_inst)
    for f in v:
        path_scr = PATH_DFS + "/" + f
        path_dest = path_inst + "/" + f
        shutil.move(path_scr, path_dest)

In [ ]:
# separando tipos de relatorios
relatorio_tipo_raw = [
    re.sub("|".join(tipo_inst.keys()), "", x[7:]).lstrip("_") for x in files
]

relatorio_tipo_raw = [
    re.sub("\\.csv|\\(SCR\\)_|\\(CNAE\\)|", "",  x) for x in relatorio_tipo_raw
]

In [ ]:
# deixando unico
relatorio_tipo_raw = list(set(relatorio_tipo_raw))

In [ ]:
# deixando unico
tipo_rel = {
    k: v for k, v in zip(
        relatorio_tipo_raw,
        [re.sub("_-__|_-_", "_", unidecode(r)).strip().lower().rstrip("_") for r in relatorio_tipo_raw]
    )
}

In [ ]:
# so tirando uma chave que deu ruim
del tipo_rel[""]

In [ ]:
# separar arquivos por tipo de instuticao
arq_por_rel = {
    short_name: [f for f in files if long_name in f]
    for long_name, short_name in tipo_rel.items()
}

In [ ]:
for v in tipo_inst.values():
    path_inst = PATH_DFS + "/" + v 
    
    for k, v in arq_por_rel.items():
        files_inst = os.listdir(path_inst)
        path_rel = path_inst + "/" + k
        os.mkdir(path_rel)
        
        for f in v:
            if f in files_inst:
                path_scr = path_inst + "/" + f
                path_dest = path_rel + "/" + f
                shutil.move(path_scr, path_dest)

In [ ]:
# reorganiza por report
root = Path(PATH_DFS)
new_root = Path(path + "/dfs_by_report")

for file in root.rglob("*.csv"):
    if file.parent.name != "legado":
        inst = file.parent.parent.name
        report = file.parent.name
        
        new_path = new_root / report / inst / file.name
        new_path.parent.mkdir(parents=True, exist_ok=True)

        file.replace(new_path)
    else:
        inst = "legado"
        report = file.stem.split("_", 1)[1]

        report = unidecode(report).lower()
        
        new_path = new_root / report / inst / file.name
        new_path.parent.mkdir(parents=True, exist_ok=True)
        
        file.replace(new_path)

In [ ]:
shutil.rmtree(path + "/dfs")